In [29]:
# Условия задачи: http://www.machinelearning.ru/wiki/index.php?title=Задача_предсказания_отклика_клиентов_ОТП_Банка_%28конкурс%29

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
plt.rc('font', family='Verdana')

In [30]:
# Увеличиваем количество отображаемых столбцов датафрейма
pd.set_option('display.max_columns', 60)


data = pd.read_csv("Credit_OTP.csv", encoding='cp1251', sep = ';')
data.head()

,AGREEMENT_RK,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,EDUCATION,MARITAL_STATUS,GEN_INDUSTRY,GEN_TITLE,ORG_TP_STATE,ORG_TP_FCAPITAL,JOB_DIR,FAMILY_INCOME,PERSONAL_INCOME,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,TP_PROVINCE,REGION_NM,REG_FACT_FL,FACT_POST_FL,REG_POST_FL,REG_FACT_POST_FL,REG_FACT_POST_TP_FL,FL_PRESENCE_FL,OWN_AUTO,AUTO_RUS_FL,HS_PRESENCE_FL,COT_PRESENCE_FL,GAR_PRESENCE_FL,LAND_PRESENCE_FL,CREDIT,TERM,FST_PAYMENT,DL_DOCUMENT_FL,GPF_DOCUMENT_FL,FACT_LIVING_TERM,WORK_TIME,FACT_PHONE_FL,REG_PHONE_FL,GEN_PHONE_FL,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_NUM_PAYM,LOAN_DLQ_NUM,LOAN_MAX_DLQ,LOAN_AVG_DLQ_AMT,LOAN_MAX_DLQ_AMT,PREVIOUS_CARD_NUM_UTILIZED
0,59910150,0,49,1,0,1,2,1,Среднее специальное,Состою в браке,Торговля,Рабочий,Частная компания,Без участия,Вспомогательный техперсонал,от 10000 до 20000 руб.,5000,Оренбургская область,Оренбургская область,Оренбургская область,Оренбургская область,ПОВОЛЖСКИЙ,1,1,1,1,1,0,0,0,0,0,0,0,8000,6,8650,0,1,220,18.0,0,0,1,1,1,6,2,1,1580,1580,NaN
1,59910230,0,32,1,0,1,3,3,Среднее,Состою в браке,Торговля,Рабочий,Индивидуальный предприниматель,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,12000,Кабардино-Балкария,Кабардино-Балкария,Кабардино-Балкария,Ставропольский край,ЮЖНЫЙ,1,1,1,1,0,0,0,0,0,0,0,0,21650,6,4000,0,1,137,97.0,1,0,1,1,1,6,1,1,4020,4020,NaN
2,59910525,0,52,1,0,1,4,0,Неполное среднее,Состою в браке,Информационные технологии,Специалист,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,9000,Иркутская область,Иркутская область,Иркутская область,Иркутская область,ВОСТОЧНО-СИБИРСКИЙ,1,1,1,1,1,0,0,0,1,0,0,0,33126,12,4000,0,1,251,84.0,0,0,1,2,1,11,0,0,0,0,NaN
3,59910803,0,39,1,0,1,1,1,Высшее,Состою в браке,Образование,Руководитель среднего звена,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 20000 до 50000 руб.,25000,Ростовская область,Ростовская область,Ростовская область,Ростовская область,ЮЖНЫЙ,0,1,0,0,1,1,0,0,0,0,0,0,"8491,82",6,5000,0,0,36,168.0,1,1,1,1,1,6,3,1,"1589,923333",1590,NaN
4,59911781,0,30,1,0,0,0,0,Среднее,Состою в браке,Государственная служба,Специалист,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,12000,Кабардино-Балкария,Кабардино-Балкария,Кабардино-Балкария,Ставропольский край,ЮЖНЫЙ,1,1,1,1,0,0,0,0,1,0,0,0,21990,12,4000,0,1,83,101.0,1,0,1,2,1,16,2,1,"1152,15",2230,NaN


In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15223 entries, 0 to 15222
Data columns (total 52 columns):
AGREEMENT_RK                  15223 non-null int64
TARGET                        15223 non-null int64
AGE                           15223 non-null int64
SOCSTATUS_WORK_FL             15223 non-null int64
SOCSTATUS_PENS_FL             15223 non-null int64
GENDER                        15223 non-null int64
CHILD_TOTAL                   15223 non-null int64
DEPENDANTS                    15223 non-null int64
EDUCATION                     15223 non-null object
MARITAL_STATUS                15223 non-null object
GEN_INDUSTRY                  13856 non-null object
GEN_TITLE                     13856 non-null object
ORG_TP_STATE                  13856 non-null object
ORG_TP_FCAPITAL               13858 non-null object
JOB_DIR                       13856 non-null object
FAMILY_INCOME                 15223 non-null object
PERSONAL_INCOME               15223 non-null object
REG_ADDRESS_PRO

In [4]:


#Заполняем пропуски
not_full_cat = ['GEN_INDUSTRY', 'GEN_TITLE', 'ORG_TP_STATE', 'ORG_TP_FCAPITAL', 'JOB_DIR']
for i in not_full_cat:
    data[i].fillna(data[i].value_counts()[0], inplace = True)
data["WORK_TIME"].fillna(data["WORK_TIME"].mean(), inplace = True)
data["PREVIOUS_CARD_NUM_UTILIZED"].fillna(0.0, inplace = True)

#Удалим ID
data.drop(['AGREEMENT_RK'], axis = 1, inplace = True)

#В некоторых столбцах цифровые данные определены, как object из-за наличия ",". Приведем к типу float
not_float = ['LOAN_AVG_DLQ_AMT', 'LOAN_MAX_DLQ_AMT', 'FST_PAYMENT', 'CREDIT', 'PERSONAL_INCOME']
for i in not_float:
    data[i] = data[i].replace(',','.', regex=True).astype(float)

In [5]:
#Создаем массивы X и y
y = data['TARGET']
X = data.drop('TARGET', axis = 1)
#Преобразуем категориальные признаки
X = pd.get_dummies(X)

In [6]:
#Создаем выборки для обучения и теста
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [7]:
#Для начала используем DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state = 42)
tree.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=42, splitter='best')

In [8]:
from sklearn.metrics import roc_auc_score

In [9]:
#Качество дерева решений (AUC)
tree2 = DecisionTreeClassifier(max_depth = 6, min_samples_leaf = 14, random_state = 42)
tree2.fit(X_train, y_train)
print(roc_auc_score(y_test, tree2.predict_proba(X_test)[:,1]))

0.64622984197


In [10]:
#Ранжируем признаки по значимости на основе DecisionTreeClassifier
feat = X.columns
best_features = []
importances = tree2.feature_importances_
ind = np.argsort(importances)[::-1]
for i in range(X.shape[1]):
    print("%d) %-s %f" % (i+1, feat[ind[i]], importances[ind[i]]))
    best_features.append(feat[ind[i]])
#print(best_features[:20])

1) PERSONAL_INCOME 0.170167
2) WORK_TIME 0.147538
3) FST_PAYMENT 0.103369
4) LOAN_AVG_DLQ_AMT 0.084976
5) LOAN_NUM_PAYM 0.083302
6) TERM 0.073289
7) PREVIOUS_CARD_NUM_UTILIZED 0.059196
8) CREDIT 0.052248
9) FACT_LIVING_TERM 0.051761
10) AGE 0.034493
11) LOAN_MAX_DLQ_AMT 0.032072
12) LOAN_NUM_CLOSED 0.031252
13) ORG_TP_STATE_Индивидуальный предприниматель 0.021244
14) EDUCATION_Высшее 0.019444
15) GEN_INDUSTRY_Государственная служба 0.016242
16) FAMILY_INCOME_от 20000 до 50000 руб. 0.010584
17) LAND_PRESENCE_FL 0.008822
18) FACT_ADDRESS_PROVINCE_Астраханская область 0.000000
19) REG_ADDRESS_PROVINCE_Ленинградская область 0.000000
20) REG_ADDRESS_PROVINCE_Карачаево-Черкесия 0.000000
21) REG_ADDRESS_PROVINCE_Карелия 0.000000
22) REG_ADDRESS_PROVINCE_Кемеровская область 0.000000
23) REG_ADDRESS_PROVINCE_Кировская область 0.000000
24) REG_ADDRESS_PROVINCE_Коми 0.000000
25) REG_ADDRESS_PROVINCE_Костромская область 0.000000
26) REG_ADDRESS_PROVINCE_Краснодарский край 0.000000
27) REG_ADDRESS_

In [15]:
#Далее будем использовать LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
#Нормализуем признаки с помощью StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state = 42)
#Сравним модели с разным типом регуляризации
logreg_l2 = LogisticRegression(random_state = 12)
logreg_l1 = LogisticRegression(random_state = 12, penalty = 'l1')
logreg_l2.fit(X_train, y_train)
logreg_l1.fit(X_train, y_train)
print('LogisticRegression(L2): ', 'AUC: ', roc_auc_score(y_test, logreg_l2.predict_proba(X_test)[:,1]))
print('LogisticRegression(L1): ', 'AUC: ', roc_auc_score(y_test, logreg_l1.predict_proba(X_test)[:,1]))

LogisticRegression(L2):  AUC:  0.685234096612
LogisticRegression(L1):  AUC:  0.689497868766


In [16]:
#Видно, что модель с L1-регуляризацией работате лучше. Будем использовать ее
#Попробуем сократить количество признаков (будут использоваться согласно таблице значимости)

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

for i in range(20, len(best_features), 40):
    X_new = X[best_features[:i]]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_new)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state = 42)
    logreg = LogisticRegression(random_state = 12, penalty = "l1")#, C=0.05)
    logreg.fit(X_train, y_train)
    print('Количество признаков: ', i, 'AUC: ', roc_auc_score(y_test, logreg.predict_proba(X_test)[:,1]))


Количество признаков:  20 AUC:  0.682615704872
Количество признаков:  60 AUC:  0.687733796622
Количество признаков:  100 AUC:  0.68720881397
Количество признаков:  140 AUC:  0.690559572814
Количество признаков:  180 AUC:  0.694036197065
Количество признаков:  220 AUC:  0.694120976872
Количество признаков:  260 AUC:  0.69531930684
Количество признаков:  300 AUC:  0.693232745353
Количество признаков:  340 AUC:  0.690101761855
Количество признаков:  380 AUC:  0.691390414925
Количество признаков:  420 AUC:  0.689641994438


In [23]:
#Видно, что в интервале 220-300 (используемых признаков) есть улучшение качества.
#Попробуем подобрать оптимальный параметр "С" для регрессии
import numpy as np
for с in np.arange(0.01, 0.1, 0.01):
    #Используем 250 параметров
    X_new = X[best_features[:250]]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_new)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state = 42)
    logreg = LogisticRegression(random_state = 12, penalty = "l1", C=с)
    logreg.fit(X_train, y_train)
    print('С: ', с, 'AUC: ', roc_auc_score(y_test, logreg.predict_proba(X_test)[:,1]))

С:  0.01 AUC:  0.682072461953
С:  0.02 AUC:  0.693875115431
С:  0.03 AUC:  0.698094541224
С:  0.04 AUC:  0.699484277911
С:  0.05 AUC:  0.700214688558
С:  0.06 AUC:  0.700680977498
С:  0.07 AUC:  0.70067054306
С:  0.08 AUC:  0.700502939903
С:  0.09 AUC:  0.700137082427


In [25]:
#Таким образом оптимальный параметр С=0.06. Качество модели улучшилось
#Сделаем еще раз более точный подбор количества признаков
for i in range(250, 350, 10):
    X_new = X[best_features[:i]]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_new)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state = 42)
    logreg = LogisticRegression(random_state = 42, penalty = "l1", C=0.06)
    logreg.fit(X_train, y_train)
    print('Количество признаков: ', i, 'AUC: ', roc_auc_score(y_test, logreg.predict_proba(X_test)[:,1]))

Количество признаков:  250 AUC:  0.700682933955
Количество признаков:  260 AUC:  0.700680651422
Количество признаков:  270 AUC:  0.700481744951
Количество признаков:  280 AUC:  0.701723769128
Количество признаков:  290 AUC:  0.70126921893
Количество признаков:  300 AUC:  0.701023357489
Количество признаков:  310 AUC:  0.701503993781
Количество признаков:  320 AUC:  0.700676412432
Количество признаков:  330 AUC:  0.700389465392
Количество признаков:  340 AUC:  0.700428594533


In [27]:
#Строим лучшую модель с параметрами: С=0.06, количество признаков = 280

logreg_final = LogisticRegression(random_state = 42, penalty = 'l1', C = 0.06)
X_280 = X[best_features[:280]]
scaler = StandardScaler()
X_scaled_280 = scaler.fit_transform(X_280)

#Проверим эту модель на кросс-валидации
from sklearn.cross_validation import StratifiedKFold, cross_val_score
strat_split = StratifiedKFold(y, n_folds = 5, random_state = 42)
score = cross_val_score(logreg_final, X_scaled_280, y, scoring = 'roc_auc', cv = strat_split)
print('AUC на кросс-валидации', score.mean())

AUC на кросс-валидации 0.68097978358
